In [3]:
import os
import cv2
import numpy as np

# Define function to load and preprocess images and masks
def load_data(train_image_dir, val_image_dir, train_mask_dir, val_mask_dir, image_size=(256, 256)):
    train_images = []
    val_images = []
    train_masks = []
    val_masks = []
    
    # Load and preprocess training images and masks
    for image_file in os.listdir(train_image_dir):
        image_path = os.path.join(train_image_dir, image_file)
        mask_path = os.path.join(train_mask_dir, image_file.replace('.JPG', '_mask.JPG'))  # Assuming masks have same filenames with '_mask' suffix
        
        # Load and preprocess image
        image = cv2.imread(image_path)
        image = cv2.resize(image, image_size)
        image = image / 255.0  # Normalize pixel values to [0, 1]
        
        train_images.append(image)
        
        # Load and preprocess mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, image_size)
        mask = mask / 255.0  # Normalize pixel values to [0, 1]
        mask = np.expand_dims(mask, axis=-1)  # Add channel dimension
        
        train_masks.append(mask)
    
    # Load and preprocess validation images and masks
    for image_file in os.listdir(val_image_dir):
        image_path = os.path.join(val_image_dir, image_file)
        mask_path = os.path.join(val_mask_dir, image_file.replace('.JPG', '_mask.JPG'))  # Assuming masks have same filenames with '_mask' suffix
        
        # Load and preprocess image
        image = cv2.imread(image_path)
        image = cv2.resize(image, image_size)
        image = image / 255.0  # Normalize pixel values to [0, 1]
        
        val_images.append(image)
        
        # Load and preprocess mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, image_size)
        mask = mask / 255.0  # Normalize pixel values to [0, 1]
        mask = np.expand_dims(mask, axis=-1)  # Add channel dimension
        
        val_masks.append(mask)
    
    return np.array(train_images), np.array(val_images), np.array(train_masks), np.array(val_masks)

# Define directories containing images and masks
train_image_dir = r'C:\Users\shanmuga\OneDrive\Documents\mini project\mini project\UDD\train\train_image'
val_image_dir = r'C:\Users\shanmuga\OneDrive\Documents\mini project\mini project\UDD\val\val_image'
train_mask_dir = r'C:\Users\shanmuga\OneDrive\Documents\mini project\mini project\UDD\train\train_mask'
val_mask_dir = r'C:\Users\shanmuga\OneDrive\Documents\mini project\mini project\UDD\val\val_mask'

# Load and preprocess the dataset
train_images, val_images, train_masks, val_masks = load_data(train_image_dir, val_image_dir, train_mask_dir, val_mask_dir)

# Display shapes of training and validation sets
print("Training images shape:", train_images.shape)
print("Validation images shape:", val_images.shape)
print("Training masks shape:", train_masks.shape)
print("Validation masks shape:", val_masks.shape)

Training images shape: (120, 256, 256, 3)
Validation images shape: (40, 256, 256, 3)
Training masks shape: (120, 256, 256, 1)
Validation masks shape: (40, 256, 256, 1)


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Define custom IoU metric
def mean_iou(y_true, y_pred):
    intersection = tf.reduce_sum(tf.cast(tf.math.logical_and(tf.math.argmax(y_true, axis=-1) == 1, tf.math.argmax(y_pred, axis=-1) == 1), dtype=tf.float32))
    union = tf.reduce_sum(tf.cast(tf.math.logical_or(tf.math.argmax(y_true, axis=-1) == 1, tf.math.argmax(y_pred, axis=-1) == 1), dtype=tf.float32))
    iou = (intersection + 1e-7) / (union + 1e-7)
    return iou

# Define custom F1 score metric
def f1_score(y_true, y_pred):
    y_true = tf.cast(tf.math.argmax(y_true, axis=-1) == 1, dtype=tf.float32)
    y_pred = tf.cast(tf.math.argmax(y_pred, axis=-1) == 1, dtype=tf.float32)
    tp = tf.reduce_sum(y_true * y_pred)
    fp = tf.reduce_sum((1 - y_true) * y_pred)
    fn = tf.reduce_sum(y_true * (1 - y_pred))
    
    # Calculate TN
    total_pixels = tf.cast(tf.size(y_true), dtype=tf.float32)
    tn = total_pixels - (tp + fp + fn)
    
    precision = tp / ( tp + fp )
    recall = tp / (tp + fn )
    f1 = 2 * (precision * recall) / (precision + recall )
    return f1

# Define the architecture with Average Pooling
def aerialsegnet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Encoder
    conv1 = layers.Conv2D(16, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(16, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.AveragePooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = layers.Conv2D(32, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(32, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.AveragePooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = layers.Conv2D(64, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.AveragePooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.AveragePooling2D(pool_size=(2, 2))(conv4)
    
    # Bottom layer
    conv5 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv5)
    
    # Decoder...
    up1 = layers.UpSampling2D(size=(2, 2))(conv5)
    up1 = layers.Conv2D(128, 2, activation='relu', padding='same')(up1)
    merge1 = layers.concatenate([conv4, up1], axis=3)
    conv6 = layers.Conv2D(128, 3, activation='relu', padding='same')(merge1)
    conv6 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv6)
    
    up2 = layers.UpSampling2D(size=(2, 2))(conv6)
    up2 = layers.Conv2D(64, 2, activation='relu', padding='same')(up2)
    merge2 = layers.concatenate([conv3, up2], axis=3)
    conv7 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge2)
    conv7 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv7)
    
    up3 = layers.UpSampling2D(size=(2, 2))(conv7)
    up3 = layers.Conv2D(32, 2, activation='relu', padding='same')(up3)
    merge3 = layers.concatenate([conv2, up3], axis=3)
    conv8 = layers.Conv2D(32, 3, activation='relu', padding='same')(merge3)
    conv8 = layers.Conv2D(32, 3, activation='relu', padding='same')(conv8)
    
    up4 = layers.UpSampling2D(size=(2, 2))(conv8)
    up4 = layers.Conv2D(16, 2, activation='relu', padding='same')(up4)
    merge4 = layers.concatenate([conv1, up4], axis=3)
    conv9 = layers.Conv2D(16, 3, activation='relu', padding='same')(merge4)
    conv9 = layers.Conv2D(16, 3, activation='relu', padding='same')(conv9)
    
    outputs = layers.Conv2D(3, 1, activation='softmax')(conv9)  # Assuming 3 classes
    
    model = models.Model(inputs=inputs, outputs=outputs)
    return model
    
    # Define outputs
    outputs = layers.Conv2D(3, 1, activation='softmax')(conv5)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Define loss function (e.g., categorical cross-entropy) and optimizer
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Compile the model with custom metrics
model = aerialsegnet(input_shape=(256, 256, 3))
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy', mean_iou, f1_score])

# Dummy data (replace with your actual dataset)
train= np.random.randn(112, 256, 256, 3)
train_mask= np.random.randint(0, 3, size=(112, 256,256))

val_image = np.random.randn(40, 256,256, 3)
test_mask = np.random.randint(0, 3, size=(40, 256, 256))

# Convert target masks to one-hot encoded format
train_mask = tf.keras.utils.to_categorical(train_mask, num_classes=3)
test_mask = tf.keras.utils.to_categorical(test_mask, num_classes=3)

# Train the model
model.fit(train, train_mask, epochs=10, validation_data=(val_image, test_mask))

# Evaluate the model
loss, accuracy, mean_iou, f1= model.evaluate(val_image, test_mask)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)
print("Mean IoU:", mean_iou)
print("F1 Score:", f1)

# Save the trained model
model.save('lw-AerialSegNet_model_.h5')

# Print the number of trainable and non-trainable parameters
num_params = np.sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
print("Number of Trainable Parameters:", num_params)


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 61s 9s/step - accuracy: 0.3331 - f1_score: 0.2924 - loss: 1.1026 - mean_iou: 0.1750 - val_accuracy: 0.3335 - val_f1_score: 0.2506 - val_loss: 1.1007 - val_mean_iou: 0.1433
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 29s 7s/step - accuracy: 0.3333 - f1_score: 0.2729 - loss: 1.1004 - mean_iou: 0.1584 - val_accuracy: 0.3331 - val_f1_score: 0.3845 - val_loss: 1.0995 - val_mean_iou: 0.2380
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 29s 7s/step - accuracy: 0.3334 - f1_score: 0.3973 - loss: 1.0994 - mean_iou: 0.2480 - val_accuracy: 0.3328 - val_f1_score: 0.3742 - val_loss: 1.0990 - val_mean_iou: 0.2302
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 41s 7s/step - accuracy: 0.3335 - f1_score: 0.2620 - loss: 1.0989 - mean_iou: 0.1564 - val_accuracy: 0.3336 - val_f1_score: 0.1777 - val_loss: 1.0987 - val_mean_iou: 0.0975
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 29s 7s/step - accuracy: 0.3336 - f1_score: 0.2894 - loss: 1.0987 - mean_iou: 0.1738 - val_accuracy: 0.3334 - val_f1_score: 0.4460 - 

Validation Loss: 1.0986146926879883
Validation Accuracy: 0.3337203860282898
Mean IoU: 0.02806304767727852
F1 Score: 0.05459398776292801
Number of Trainable Parameters: 1941139


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Define function to preprocess input image
def preprocess_image(image, target_size=(256, 256)):
    # Resize image to match model input size
    resized_image = cv2.resize(image, target_size)
    
    # Normalize pixel values to be within [0, 1]
    normalized_image = resized_image / 255.0
    
    return normalized_image

# Load the pre-trained model
model = load_model('LW-AerialSegNet_model_.h5')

# Define the color mappings for different classes
class_colors = {
    0: [255, 0, 0],  
    1: [0, 0, 0],    
    2: [0, 255, 0]   
}

def segment_image(image, model):
    # Preprocess the input image
    preprocessed_image = preprocess_image(image)
    
    # Perform segmentation using the U-Net model
    segmented_image = model.predict(np.expand_dims(preprocessed_image, axis=0))[0]

    # Convert predicted segmentation mask to class labels
    segmented_image = np.argmax(segmented_image, axis=-1)
    
    # Resize the segmentation mask to match the original image size
    resized_segmented_image = cv2.resize(segmented_image.astype(np.uint8), (image.shape[1], image.shape[0]))
    
    # Convert class labels to color-coded segmentation mask
    colored_mask = np.zeros_like(image)
    for class_id, color in class_colors.items():
        colored_mask[resized_segmented_image == class_id] = color

    return colored_mask

# Load input image
image = cv2.imread(r'C:\Users\shanmuga\OneDrive\Documents\mini project\mini project\UDD\val\val_mask\DJI_0303_mask.JPG')

# Segment the input image
segmented_image = segment_image(image, model)

# Display the input and segmented images
cv2.namedWindow('Input Image', cv2.WINDOW_NORMAL)
cv2.imshow('Input Image', image)

cv2.namedWindow('Segmented Image', cv2.WINDOW_NORMAL)
cv2.imshow('Segmented Image', segmented_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 700ms/step
